In [14]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, sigmay
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp

N = 16
hi = nk.hilbert.Spin(s=1 / 2, N=N)
theta1 = 0.25+0.0j
theta2 = 0.25+0.0j
H = 2*sum([sigmax(hi, i)*sigmax(hi, i+1)+sigmay(hi, i)*sigmay(hi, i+1) for i in range(1, int(N/2))])
H += 2*sum([sigmam(hi, i)*sigmap(hi, (i+1)%N)+sigmap(hi, i)*sigmam(hi, (i+1)%N) for i in range(int(N/2+1), N)])
H += 2*jnp.cos(theta1)*(sigmax(hi, 0)*sigmax(hi, 1)+sigmay(hi, 0)*sigmay(hi, 1))+jnp.sin(theta1)*(sigmax(hi, 0)*sigmay(hi, 1)-sigmay(hi, 0)*sigmax(hi, 1))
H += 2*jnp.cos(theta2)*(sigmax(hi, int(N/2))*sigmax(hi, int(N/2)+1)+sigmay(hi, int(N/2))*sigmay(hi, int(N/2)+1))+jnp.sin(theta2)*(sigmax(hi, int(N/2))*sigmay(hi, int(N/2)+1)-sigmay(hi, int(N/2))*sigmax(hi, int(N/2)+1))
H += sum([sigmaz(hi,i)*sigmaz(hi,(i+1)%N) for i in range(N)])
sp_h=H.to_sparse()
eig_vals, eig_vecs = eigsh(sp_h, k=2, which="SA")

print("eigenvalues with scipy sparse:", eig_vals)

E_gs = eig_vals[0]

eigenvalues with scipy sparse: [-39.68304359 -38.43108525]


In [1]:
import netket as nk
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, sigmay
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp


In [33]:
import netket as nk

# Define the Hilbert space, typically for a spin system
hilbert_space = nk.hilbert.Spin(s=0.5, N=1)  # Example for a single spin-1/2 system

# Define the σ^x and σ^z operators
sigmax = nk.operator.spin.sigmax(hilbert=hilbert_space, site=0)
sigmaz = nk.operator.spin.sigmaz(hilbert=hilbert_space, site=0)

# Convert operators to dense matrix form to access elements
matrix_sigmax = (identity(hilbert_space)-sigmax).to_dense()
matrix_sigmaz = sigmaz.to_dense()
print(matrix_sigmax)

[[ 1. -1.]
 [-1.  1.]]


In [102]:
import jax
import netket as nk
import numpy as np
from netket.operator.spin import sigmax,sigmaz, sigmap, sigmam, identity
import time
from scipy.sparse.linalg import eigsh
import jax.numpy as jnp
import matplotlib.pyplot as plt
from jax.random import PRNGKey, categorical, split
from twoD_tool import *
Lx = 4
Ly = 3
N = Lx * Ly
params = [ 0.4, 0.8, 1.0 ,1.2, 1.4, 1.6, 1.8, 2.0, 2.4, 2.8, 3.2]
Omega = 1.0
Rb = 1.2**6
hi = nk.hilbert.Spin(s=1 / 2, N =  N)
numsamples = 1024
batch_categorical = jax.vmap(categorical, in_axes=(0, None))
batch_dot = jax.vmap(jnp.dot, (0, None))
key = PRNGKey(1)
for param in params:
    key, subkey = split(key, 2)
    key_ = split(subkey, numsamples)
    ni = []
    H = Omega/2*sum([sigmax(hi, y*Lx+x) for y in range (Ly) for x in range (Lx)]) #X
    H -= param/2*sum([(identity(hi)-sigmaz(hi, y*Lx+x)) for y in range(Ly) for x in range(Lx)])
    H += Omega*Rb/4*sum([((identity(hi)-sigmaz(hi, y1*Lx+x1))*(identity(hi)-sigmaz(hi, y1*Lx+x2)))/((x1-x2)**2)**3 \
                         for y1 in range(Ly) for x1 in range(Lx) for x2 in range(x1+1, Lx)])
    H += Omega*Rb/4*sum([((identity(hi)-sigmaz(hi, y1*Lx+x1))*(identity(hi)-sigmaz(hi, y2*Lx+x2)))/(((x1-x2)**2+(y1-y2)**2)**3) \
                         for y1 in range(Ly) for x1 in range(Lx) for y2 in range(y1+1, Ly) for x2 in range(Lx)])
    sp_h = H.to_sparse()
    eig_vals, eig_vecs = eigsh(sp_h, k=3, which="SA")
    
    samples = batch_categorical(key_, jnp.log(jnp.real(eig_vecs[:, 0].conj()*eig_vecs[:, 0])))
    samples_b = jnp.flip(jnp.unpackbits(samples.view('uint8'), bitorder='little').reshape(numsamples, -1), axis = 1)[:, 48:]
    alt_m = create_alternating_matrix(Lx)
    stagger_mag = jnp.mean(jnp.abs(batch_dot(samples_b.reshape(numsamples, -1), create_alternating_matrix(Lx).ravel().T))/(Lx*Ly))
    '''
    for y in range(Ly):
        for x in range(Lx):
            h_ni = (identity(hi)-sigmaz(hi, y*Lx+x))/2
            ni.append(eig_vecs[:,0].conj() @ h_ni.to_sparse() @ eig_vecs[:,0])
    
    stagger_H_sp = stagger_H.to_sparse()
    stagger_mag0 = np.abs(eig_vecs[:,0].conj() @ stagger_H_sp @ eig_vecs[:,0])
    stagger_mag1 = np.abs(eig_vecs[:,0].conj() @ stagger_H_sp @ eig_vecs[:,1])
    stagger_mag2 = np.abs(eig_vecs[:,1].conj() @ stagger_H_sp @ eig_vecs[:,1])
    stagger_mag3 = np.abs(eig_vecs[:,0].conj() @ stagger_H_sp @ eig_vecs[:,2])
    stagger_mag4 = np.abs(eig_vecs[:,1].conj() @ stagger_H_sp @ eig_vecs[:,2])
    stagger_mag5 = np.abs(eig_vecs[:,2].conj() @ stagger_H_sp @ eig_vecs[:,2])
    print(stagger_mag0, stagger_mag1, stagger_mag2, stagger_mag3, stagger_mag4, stagger_mag5)
    '''
    print(param, stagger_mag)

0.4 0.12573242
0.8 0.1508789
1.0 0.16088867
1.2 0.177653
1.4 0.20491536
1.6 0.23543295
1.8 0.27734375
2.0 0.3330078
2.4 0.42000324
2.8 0.45157877
3.2 0.45613605


In [54]:
hi = nk.hilbert.Spin(s=1 / 2, N =  1)
H = identity(hi)-sigmaz(hi, 0)
H.to_sparse() @ [1, 0]

array([0., 0.])

(512, 1)

In [61]:

create_alternating_matrix(4)

array([[ 1, -1,  1, -1],
       [-1,  1, -1,  1],
       [ 1, -1,  1, -1],
       [-1,  1, -1,  1]])

In [95]:
jnp.flip(jnp.unpackbits(jnp.array([4096,]).view('uint8'), bitorder='little'))[48:]

Array([0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], dtype=uint8)